### setup libraries

In [184]:
import os, time, json
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from datetime import datetime
from geotext import GeoText
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType

### define selenium driver

In [185]:
# Set the path to the Brave browser executable
brave_path = r'C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe'
# Launch Brave browser using Selenium
options = webdriver.ChromeOptions()
options.binary_location = brave_path

driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()),
                         options=options)

### login to linkedin

In [186]:
user_email = 'djdjoko@gmail.com'
user_pwd = 'PItuUT^pYp7c3WvZ!871hziDF'

# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")
 
# waiting for the page to load
time.sleep(5)
 
# entering username
username = driver.find_element(By.ID, "username")
 
# In case of an error, try changing the element
# tag used here.
 
# Enter Your Email Address
username.send_keys(user_email) 
 
# entering password
pword = driver.find_element(By.ID, "password")
# In case of an error, try changing the element
# tag used here.
 
# Enter Your Password
pword.send_keys(user_pwd)       
 
# Clicking on the log in button
# Format (syntax) of writing XPath -->
# //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()
# In case of an error, try changing the
# XPath used here.

In [187]:
def scroll_down(driver):
    # Wait for page to load
    time.sleep(5)

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for the page to load
        time.sleep(2)

        # Calculate the new page height and compare with the previous height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [188]:
from fuzzywuzzy import fuzz, process

# search dictionary to see if the organization is existing
def min_case(str_full, n_max = 5, separator = ' '):
    str_split = str_full.strip().split(separator)
    new_str = ''
    for s in str_split:
        if new_str == '':
            new_str = s
        else:
            new_str = new_str + separator + s
        n_case = len(new_str)
        if n_case >= n_max:
            return new_str
    return new_str

def fuzzy_organization_match(d, match_value, min_case_num=5):
    match_value = min_case(match_value, min_case_num).lower()
    for x in experience_list:
        organization = min_case(x['Organization'], min_case_num).lower()
        ratio = fuzz.token_sort_ratio(organization, match_value)
#         ratio = fuzz.ratio(x['Organization'].lower(), match_value.lower())
        print(organization, match_value, ratio)
        if ratio >= 80:  # threshold for matching
            return True

    return False

x = 'made.com design ltd'
y = 'made.com design limited'
x_min = min_case(x)
y_min = min_case(y)
print('comparing x vs y:', x, y, fuzz.partial_ratio(x, y))
print('comparing x_min vs y_min:', x_min, y_min, fuzz.partial_ratio(x_min, y_min))


comparing x vs y: made.com design ltd made.com design limited 89
comparing x_min vs y_min: made.com made.com 100


In [189]:
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, '%b %Y')
    except ValueError:
        try:
            return datetime.strptime(date_str, '%Y')
        except ValueError:
            raise ValueError(f"Invalid date string: '{date_str}'")

In [190]:
def set_period(span, d, d_item):
    date_range = span.get_text().split('·')[0]
    if '-' in date_range: 
        fr, to = date_range.split('-')
        fr_dt = parse_date(fr.strip())
    else:
        fr_dt = parse_date(date_range.strip())
        to = fr_dt.replace(month=12, day=31).strftime('%b %Y')

    if to.strip().lower() == 'present':
        to_dt = datetime.now()
    else:
        to_dt = parse_date(to.strip())
    
    if d['From'] == '':
        fr_dt_min = datetime.max
    else:
        fr_dt_min = datetime.strptime(d['From'],'%Y-%m-%d')
        
    if d['To'] == '':
        to_dt_max = datetime.min
    else:
        to_dt_max = datetime.strptime(d['To'],'%Y-%m-%d')
        
    fr_dt_min = min(fr_dt, fr_dt_min)
    to_dt_max = max(to_dt, to_dt_max)
    period_minmax = to_dt_max - fr_dt_min

    d['From'] = fr_dt_min.strftime('%Y-%m-%d')
    d['To'] = to_dt_max.strftime('%Y-%m-%d')
    d['Days'] = period_minmax.days

    period = to_dt - fr_dt
    d_item['From'] = fr_dt.strftime('%Y-%m-%d')
    d_item['To'] = to_dt.strftime('%Y-%m-%d')
    d_item['Days'] = period.days
    

def set_location(span, d, d_item):
    location_text = span.get_text()
    location = GeoText(location_text)
    d_item['Country'] = "" if len(location.country_mentions) == 0 else list(location.country_mentions.keys())[0]
    d_item['City'] = "" if len(location.cities) == 0 else location.cities[0]

def set_item(span, name, d):
    d[name] = span.get_text().split('·')[0].strip()
    
def set_dict(s, s_type_prev, parse_type, d, d_item):
    s_class = s.parent.get('class')
#     print(s, s_class)
    if 'mr1' in s_class and 't-bold' in s_class: # this indicates a Position or Organization (education)
        if parse_type == 'experience':
            s_type = 'Position'
            set_item(s, s_type, d_item)
        elif parse_type == 'education':
            s_type = 'Organization'
            set_item(s, s_type, d)
        else:
            raise NotImplementedError(str(s))

    elif 't-14' in s_class and 't-normal' in s_class and 't-black--light' in s_class: # indicates Period or Location
        if s_type_prev == 'Period':
            s_type = 'Location'
            set_location(s, d, d_item)
        else:
            s_type = 'Period'
            set_period(s, d, d_item)

    elif 't-14' in s_class and 't-normal' in s_class and 't_black--light' not in s_class and 'display-flex' not in s_class: # this indicates an Organization (experience) or Degree
        if parse_type == 'experience':
            s_type = 'Organization'
            set_item(s, s_type, d)
        elif parse_type == 'education':
            s_type = 'Degree'
            set_item(s, s_type, d_item)
        else:
            raise NotImplementedError(str(s))


    elif 'display-flex' in s_class: # indicated Description
        s_type = 'Description'
        set_item(s, s_type, d_item)

    else:
        s_type = s_type_prev
        print('Following item is not set:', str(s_class), str(s))
    
    return s_type

def parse_list(html_lst, parse_type, verbose=False):
#     keys = ['Position', 'Organization', 'Period', 'Description'] # parse experience
#     keys = ['Organization', 'Degree', 'Period', 'Description'] # parse education
    lst = []
    #extract text elements
    element_num = 0
    for item in html_lst:
        d = {}
        d['items'] = []

        d['Organization'] = ''
        d['From'] = ''
        d['To'] = ''
        d['Days'] = ''
        
        span = item.find_all('span', attrs={'aria-hidden': 'true'})
        soup_list = BeautifulSoup(str(html_lst[element_num]), 'html.parser')
        
        if soup_list.find('ul', {'class': 'pvs-list'}) is None or soup_list.find('ul', {'class': 'pvs-list'}).find('ul', {'class': 'pvs-list'}) is None:
            soup_sublist = [None]
        else:
            soup_sublist = soup_list.find('ul', {'class': 'pvs-list'}).find('ul', {'class': 'pvs-list'}).find_all('li', recursive=False)
            
        subelement_num = 0
        fr_dt_min = datetime.max
        to_dt_max = datetime.min
        for sublist in soup_sublist:
#             print(element_num, subelement_num)
            d_item = {}
            
            if parse_type == 'experience':
                d_item['Position'] = ''
            elif parse_type == 'education':
                d_item['Degree'] = ''
            else:
                raise NotImplementedError()

            d_item['From'] = ''
            d_item['To'] = ''
            d_item['Days'] = ''
            d_item['Country'] = ''
            d_item['City'] = ''
            d_item['Description'] = ''
        
            if len(soup_sublist) > 1:
                span_sublist = soup_sublist[subelement_num].find_all('span', attrs={'aria-hidden': 'true'})
            
            subelement_num += 1
            
            if len(soup_sublist) == 1:
                s_type = ''
                for s in span:
                    s_type = set_dict(s, s_type, parse_type, d, d_item)

                
            elif len(soup_sublist) >= 2:
                if parse_type == 'experience':
                    d_item['Position'] = span_sublist[0].get_text().split('·')[0].strip()
                    d['Organization'] = span[0].get_text().split('·')[0].strip()
                elif parse_type == 'education':
                    d_item['Degree'] = span_sublist[1].get_text().split('·')[0].strip()
                    d['Organization'] = span[0].get_text().split('·')[0].strip()
                else:
                    raise NotImplementedError()
                
                s_type = ''
                # period
                for i in range(1, len(span_sublist)):
                    s = span_sublist[i]
                    s_type = set_dict(s, s_type, parse_type, d, d_item)
            
#                 print(s.get_text(), s_class)
                
            d['items'].append(d_item)
            
#         d[keys[0]] = span[0].get_text()
#         d[keys[1]] = span[1].get_text().split('·')[0].strip()

        lst.append(d)
        if verbose:
            print(json.dumps(d, indent=4))
        element_num += 1
        
    return lst

In [191]:
profile_url = 'https://www.linkedin.com/in/djordje-djokovic-phd-683589/' 
profile_url = 'https://www.linkedin.com/in/branko-jevtic-752a008/'
profile_url = 'https://www.linkedin.com/in/lining/'
profile_url = 'https://www.linkedin.com/in/juliencallede/'
profile_url = 'https://www.linkedin.com/in/philippe-chainieux-2b4812/'
profile_url = 'https://www.linkedin.com/in/adrian-evans-a406b96/'
profile_url = 'https://www.linkedin.com/in/bengothard/' # check 
profile_url = 'https://www.linkedin.com/in/brenthoberman/'
profile_url = 'https://www.linkedin.com/in/lisa-gan-tomlins-8a660a3b/'

### get info from main page

In [192]:
driver.get(profile_url)
# scroll_down(driver)
src = driver.page_source
# Now using beautiful soup
soup = BeautifulSoup(src, 'lxml')

# Extracting the HTML of the complete introduction box
# that contains the name, company name, and the location
intro = soup.find_all('div', {'class': 'pv-text-details__left-panel'})

# In case of an error, try changing the tags used here.
 
name_loc = intro[0].find("h1")
 
# Extracting the Name
name = name_loc.get_text().strip()
# strip() is used to remove any extra blank spaces
 
works_at_loc = intro[0].find("div", {'class': 'text-body-medium'})
 
# this gives us the HTML of the tag in which the Company Name is present
# Extracting the Company Name
works_at = works_at_loc.get_text().strip()

location_loc = intro[1].find("span", {'class': 'text-body-small'})
 
# Ectracting the Location
# The 2nd element in the location_loc variable has the location
location_text = location_loc.get_text().strip()
 
info = {}
info['Name'] = name
# info['Company'] = name

location = GeoText(location_text)
info['Country'] = "" if len(location.country_mentions) == 0 else list(location.country_mentions.keys())[0]
info['City'] = "" if len(location.cities) == 0 else location.cities[0]
    

print(info)

{'Name': 'Lisa Gan Tomlins', 'Country': 'GB', 'City': 'London'}


### get experience

In [81]:
driver.get(profile_url + 'details/experience/')
scroll_down(driver)
src = driver.page_source
# Now using beautiful soup
soup = BeautifulSoup(src, 'lxml')

#recursive false is needed otherwise sublists will be extracted as well
experience_html_list = soup.find('ul', {'class': 'pvs-list'}).find_all('li', recursive=False)

experience_list = parse_list(experience_html_list, 'experience', True)

{
    "items": [
        {
            "Position": "Co-Founder and Executive Chair",
            "From": "2006-01-01",
            "To": "2023-05-05",
            "Days": 6333,
            "Country": "GB",
            "City": "London",
            "Description": "Founders Forum Group is an event-powered community and group of businesses supporting entrepreneurs at every stage of their journeys. Its invite-only forums unite the world\u2019s most influential founders, investors, corporate and government leaders to tackle era-defining questions in iconic locations across the globe.The group launches multiple service businesses to support founders scale their journey. Since 2019, Founders Forum has partnered with Informa Tech to celebrate the strength and diversity of UK tech through London Tech Week. The joint venture allows LTW to enrich its impact with sector-specific and diversity-driven events including ClimateTech, Elevating Founders, EQL Her, and HealthTech."
        }
    ],
    "O

In [163]:
fuzzy_organization_match(experience_list, 'made.com', 5)

founders made.com 38
founders made.com 38
firstminute made.com 21
london made.com 29
founders made.com 38
founders made.com 38
founders made.com 38
google made.com 29
great made.com 15
tessa made.com 15
karakuri made.com 12
planet made.com 29
founders made.com 38
founders made.com 38
01 founders made.com 32
grip - made.com 0
eql:her made.com 27
founders made.com 38
angel made.com 31
royal made.com 31
imperial made.com 25
founders made.com 38
made.com made.com 100


True

In [ ]:
# # this is just for testing purposes

# # this is the first list containing the company.  
# element_num = 1
# # this is the second list within a company that can contain multiple job positions withing a company. 
# # the default is that the element_num has always only one element. havent come across more then one
# # the default is also that the subelement has one element. if the subelement has multiple elements this means that the profile has multiple positions within a company
# # the structure is different if there is only one subelement and more then one subelement. both cases need to be handled differently
# subelement_num = 0

# soup_list = BeautifulSoup(str(experience_html_list[element_num]), 'html.parser')
# soup_sublist = soup_list.find('ul', {'class': 'pvs-list'}).find('ul', {'class': 'pvs-list'}).find_all('li', recursive=False)
# soup_list.find_all('span', attrs={'aria-hidden': 'true'})
# # soup_sublist[subelement_num].find_all('span', attrs={'aria-hidden': 'true'})

# # if subelements length is = 1
# # index 0 = position
# # index 1 = organization
# # index 2 = from to · period
# # index 3 = location
# # index 4 = description

# # if subelements length is = 2
# # index 0 = organization
# # index 1 = period
# # subindex 1 = position 
# # subindex 2 = from to · period
# # subindex 3 = location
# # subindex 4 = description

### get education

In [83]:
driver.get(profile_url + 'details/education/')
scroll_down(driver)
src = driver.page_source
# Now using beautiful soup
soup = BeautifulSoup(src, 'lxml')

#recursive false is needed otherwise sublists will be extracted as well
education_html_list = soup.find('ul', {'class': 'pvs-list'}).find_all('li', recursive=False)

education_list = parse_list(education_html_list, 'education', True)

Following item is not set: ['inline-show-more-text', 'inline-show-more-text--is-collapsed', 'inline-show-more-text--is-collapsed-with-line-clamp', 'full-width'] <span aria-hidden="true"><!-- -->Activities and societies: President The French Club<!-- --></span>
{
    "items": [
        {
            "Degree": "Master of Arts - MA, Modern Languages & Literature",
            "From": "1988-01-01",
            "To": "1992-01-01",
            "Days": 1461,
            "Country": "",
            "City": "",
            "Description": "New College"
        }
    ],
    "Organization": "University of Oxford",
    "From": "1988-01-01",
    "To": "1992-01-01",
    "Days": 1461
}
Following item is not set: ['inline-show-more-text', 'inline-show-more-text--is-collapsed', 'inline-show-more-text--is-collapsed-with-line-clamp', 'full-width'] <span aria-hidden="true"><!-- -->Grade: Oppidan Scholar<!-- --></span>
Following item is not set: ['inline-show-more-text', 'inline-show-more-text--is-collapsed'

### remove duplicates

In [246]:
l = []
l.append({"info": {"name": "Carson Roen", "full_name": "ROEN, Carson", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "GB", "city": "London"}, "experience": [{"items": [{"position": "Marketing", "from": "2017-01-01", "to": "2023-05-08", "days": 2318, "country": "GB", "city": "London", "description": ""}], "organization": "Strike", "from": "2017-01-01", "to": "2023-05-08", "days": 2318}, {"items": [{"position": "Marketing", "from": "2013-10-01", "to": "2016-02-01", "days": 853, "country": "GB", "city": "London", "description": "Performance marketing & advertising, measurement, analysis, brand building, market positioning, comms strategy, strategic business development, monetization/optimization and all things marketing to ensure the business continues to grow in a fast, efficient and sustainable way. General management and leadership beyond marketing including: business opportunity capitalisation (identify key opportunity for resource, develop solutions, implement, measure, iterate if still largest opportunity), organsational improvements, policy development, fulfilment/logistics support, data/tech support, customer service guidance."}], "organization": "Clippings.com", "from": "2013-10-01", "to": "2016-02-01", "days": 853}, {"items": [{"position": "Marketing", "from": "2010-06-01", "to": "2013-10-01", "days": 1218, "country": "GB", "city": "London", "description": "Performance/ROI driven marketing - Online (search engine marketing, search engine optimization, digital display, remarketing, social media, outreach, product feed syndication marketing, affiliate marketing, email marketing, referral marketing), Offline (national and consumer press display and advetorial advertising, partnerships, direct response), Optimization (onsite merchandising, iterative marketing campaigns), Innovation (mobile/tablet site, utilizing latest best practices and quickly capitalizing on marketing opportunities)Data management, analysis and utilization -  marketing channels performance/attribution, product performance, user interface performance, customer lifetime value analysis, customer socio-geo-demographic profiling"}], "organization": "Made.com", "from": "2010-06-01", "to": "2013-10-01", "days": 1218}, {"items": [{"position": "Marketing", "from": "2008-11-01", "to": "2010-01-01", "days": 426, "country": "US", "city": "San Francisco", "description": "Director of Growth & Marketing"}], "organization": "Academia.edu", "from": "2008-11-01", "to": "2010-01-01", "days": 426}, {"items": [{"position": "Content", "from": "2008-04-01", "to": "2008-08-01", "days": 122, "country": "US", "city": "San Francisco", "description": "Travel Content Curator"}], "organization": "Offbeat Guides", "from": "2008-04-01", "to": "2008-08-01", "days": 122}, {"items": [{"position": "Adventurer", "from": "2006-06-01", "to": "2007-12-01", "days": 548, "country": "", "city": "", "description": "The best thing I have ever done in my life."}], "organization": "eBUS live", "from": "2006-06-01", "to": "2007-12-01", "days": 548}], "education": [{"items": [{"degree": "Bachelor of Science (B.S.), Business Administration", "from": "2002-01-01", "to": "2006-01-01", "days": 1461, "country": "", "city": "", "description": ""}], "organization": "Pepperdine University", "from": "2002-01-01", "to": "2006-01-01", "days": 1461}]})
l.append({"info": {"name": "Carson Roen", "full_name": "ROEN, Carston", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "GB", "city": "London"}, "experience": [{"items": [{"position": "Marketing", "from": "2017-01-01", "to": "2023-05-08", "days": 2318, "country": "GB", "city": "London", "description": ""}], "organization": "Strike", "from": "2017-01-01", "to": "2023-05-08", "days": 2318}, {"items": [{"position": "Marketing", "from": "2013-10-01", "to": "2016-02-01", "days": 853, "country": "GB", "city": "London", "description": "Performance marketing & advertising, measurement, analysis, brand building, market positioning, comms strategy, strategic business development, monetization/optimization and all things marketing to ensure the business continues to grow in a fast, efficient and sustainable way. General management and leadership beyond marketing including: business opportunity capitalisation (identify key opportunity for resource, develop solutions, implement, measure, iterate if still largest opportunity), organsational improvements, policy development, fulfilment/logistics support, data/tech support, customer service guidance."}], "organization": "Clippings.com", "from": "2013-10-01", "to": "2016-02-01", "days": 853}, {"items": [{"position": "Marketing", "from": "2010-06-01", "to": "2013-10-01", "days": 1218, "country": "GB", "city": "London", "description": "Performance/ROI driven marketing - Online (search engine marketing, search engine optimization, digital display, remarketing, social media, outreach, product feed syndication marketing, affiliate marketing, email marketing, referral marketing), Offline (national and consumer press display and advetorial advertising, partnerships, direct response), Optimization (onsite merchandising, iterative marketing campaigns), Innovation (mobile/tablet site, utilizing latest best practices and quickly capitalizing on marketing opportunities)Data management, analysis and utilization -  marketing channels performance/attribution, product performance, user interface performance, customer lifetime value analysis, customer socio-geo-demographic profiling"}], "organization": "Made.com", "from": "2010-06-01", "to": "2013-10-01", "days": 1218}, {"items": [{"position": "Marketing", "from": "2008-11-01", "to": "2010-01-01", "days": 426, "country": "US", "city": "San Francisco", "description": "Director of Growth & Marketing"}], "organization": "Academia.edu", "from": "2008-11-01", "to": "2010-01-01", "days": 426}, {"items": [{"position": "Content", "from": "2008-04-01", "to": "2008-08-01", "days": 122, "country": "US", "city": "San Francisco", "description": "Travel Content Curator"}], "organization": "Offbeat Guides", "from": "2008-04-01", "to": "2008-08-01", "days": 122}, {"items": [{"position": "Adventurer", "from": "2006-06-01", "to": "2007-12-01", "days": 548, "country": "", "city": "", "description": "The best thing I have ever done in my life."}], "organization": "eBUS live", "from": "2006-06-01", "to": "2007-12-01", "days": 548}], "education": [{"items": [{"degree": "Bachelor of Science (B.S.), Business Administration", "from": "2002-01-01", "to": "2006-01-01", "days": 1461, "country": "", "city": "", "description": ""}], "organization": "Pepperdine University", "from": "2002-01-01", "to": "2006-01-01", "days": 1461}]})
l.append({"info": {"name": "Carson Roen", "full_name": "ROEN, Carstons", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "GB", "city": "London"}, "experience": [{"items": [{"position": "Marketing", "from": "2017-01-01", "to": "2023-05-08", "days": 2318, "country": "GB", "city": "London", "description": ""}], "organization": "Strike", "from": "2017-01-01", "to": "2023-05-08", "days": 2318}, {"items": [{"position": "Marketing", "from": "2013-10-01", "to": "2016-02-01", "days": 853, "country": "GB", "city": "London", "description": "Performance marketing & advertising, measurement, analysis, brand building, market positioning, comms strategy, strategic business development, monetization/optimization and all things marketing to ensure the business continues to grow in a fast, efficient and sustainable way. General management and leadership beyond marketing including: business opportunity capitalisation (identify key opportunity for resource, develop solutions, implement, measure, iterate if still largest opportunity), organsational improvements, policy development, fulfilment/logistics support, data/tech support, customer service guidance."}], "organization": "Clippings.com", "from": "2013-10-01", "to": "2016-02-01", "days": 853}, {"items": [{"position": "Marketing", "from": "2010-06-01", "to": "2013-10-01", "days": 1218, "country": "GB", "city": "London", "description": "Performance/ROI driven marketing - Online (search engine marketing, search engine optimization, digital display, remarketing, social media, outreach, product feed syndication marketing, affiliate marketing, email marketing, referral marketing), Offline (national and consumer press display and advetorial advertising, partnerships, direct response), Optimization (onsite merchandising, iterative marketing campaigns), Innovation (mobile/tablet site, utilizing latest best practices and quickly capitalizing on marketing opportunities)Data management, analysis and utilization -  marketing channels performance/attribution, product performance, user interface performance, customer lifetime value analysis, customer socio-geo-demographic profiling"}], "organization": "Made.com", "from": "2010-06-01", "to": "2013-10-01", "days": 1218}, {"items": [{"position": "Marketing", "from": "2008-11-01", "to": "2010-01-01", "days": 426, "country": "US", "city": "San Francisco", "description": "Director of Growth & Marketing"}], "organization": "Academia.edu", "from": "2008-11-01", "to": "2010-01-01", "days": 426}, {"items": [{"position": "Content", "from": "2008-04-01", "to": "2008-08-01", "days": 122, "country": "US", "city": "San Francisco", "description": "Travel Content Curator"}], "organization": "Offbeat Guides", "from": "2008-04-01", "to": "2008-08-01", "days": 122}, {"items": [{"position": "Adventurer", "from": "2006-06-01", "to": "2007-12-01", "days": 548, "country": "", "city": "", "description": "The best thing I have ever done in my life."}], "organization": "eBUS live", "from": "2006-06-01", "to": "2007-12-01", "days": 548}], "education": [{"items": [{"degree": "Bachelor of Science (B.S.), Business Administration", "from": "2002-01-01", "to": "2006-01-01", "days": 1461, "country": "", "city": "", "description": ""}], "organization": "Pepperdine University", "from": "2002-01-01", "to": "2006-01-01", "days": 1461}]})
l.append({"info": {"name": "Ning Li", "full_name": "SIMONCINI, Marc", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "", "city": ""}, "experience": [{"items": [{"position": "Founder", "from": "2019-01-01", "to": "2023-05-08", "days": 1588, "country": "FR", "city": "", "description": "Typology is a Paris based skincare brand created on a quest to the right ingredients for the skin.Launched in Feb 2019 as a B-corp at inception, Typology was also the #1 selling skincare brand online in France in 2021 according to an independent research.Reach out to us if you are interested in joining a team - all available positions + spontaneous applications available here https://www.typology.com/rejoindre-lequipe"}], "organization": "Typology", "from": "2019-01-01", "to": "2023-05-08", "days": 1588}, {"items": [{"position": "non exec board director", "from": "2017-02-01", "to": "2022-11-01", "days": 2099, "country": "GB", "city": "", "description": "Founder of MADE.COM.Since the IPO in the London Stock Exchange I remain a non executive director at the board of Made PLC."}, {"position": "co-founder & CEO", "from": "2009-11-01", "to": "2017-01-01", "days": 2618, "country": "GB", "city": "London", "description": "Founder of MADE.COM - a London based e-commerce brand that designs, manufactures and retails high-end home designs at affordable price for European consumers. I served as cofounder and Chief Executive for the first 7 years from inception of 2010. In 2017, I stepped back as vice Chairman of the Board whilst remaining a significant shareholder. Made.com went IPO in July 2021 in the London Stock Exchange."}], "organization": "MADE.COM", "from": "2009-11-01", "to": "2022-11-01", "days": 4748}, {"items": [{"position": "co-founder & CEO", "from": "2007-06-01", "to": "2009-05-01", "days": 700, "country": "FR", "city": "", "description": "www.myfab.comI co-founded Myfab and served as CEO until 2009.  Myfab was one of the first private sale businesses in decor/gadget category and was head-quartered in Paris. The business employed over 180 staff and it was partially sold to Pinault Printemps Redoute (now Kering), one of the leading global retail/luxury group in 2009."}], "organization": "MYFAB", "from": "2007-06-01", "to": "2009-05-01", "days": 700}, {"items": [{"position": "assistant to the founder", "from": "2006-06-01", "to": "2007-05-01", "days": 334, "country": "FR", "city": "", "description": "First as an intern then a full time position, I served as assistant to Marc Simoncini, founder and CEO of MEETIC, for various missions of corporate development projects (M&A, key contract negotiations, and country management)."}], "organization": "Meetic", "from": "2006-06-01", "to": "2007-05-01", "days": 334}, {"items": [{"position": "Intern", "from": "2004-07-01", "to": "2005-08-01", "days": 396, "country": "FR", "city": "Paris", "description": "Placement year with investment bank Rothschild & Co in Paris and Hong Kong office.  M&A and Equity Capital Market internship placement during the 2004-2005 school year."}], "organization": "Rothschild & Co", "from": "2004-07-01", "to": "2005-08-01", "days": 396}, {"items": [{"position": "Intern", "from": "2004-05-01", "to": "2004-07-01", "days": 61, "country": "", "city": "", "description": ""}], "organization": "Boston Consulting Group (BCG)", "from": "2004-05-01", "to": "2004-07-01", "days": 61}], "education": [{"items": [{"degree": "Master of Science, Majeure Entrepreneurs", "from": "2003-01-01", "to": "2006-01-01", "days": 1096, "country": "", "city": "", "description": "Paris, France"}], "organization": "HEC Paris", "from": "2003-01-01", "to": "2006-01-01", "days": 1096}]})
l.append({"info": {"name": "Andy Skipper", "full_name": "SKIPPER, Andrew", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "", "city": ""}, "experience": [{"items": [{"position": "Founder and Chief CTO Coach", "from": "2017-05-01", "to": "2023-05-08", "days": 2198, "country": "GB", "city": "London", "description": "Our mission is to empower senior engineering leaders in startups and scale-ups to continuously learn, grow, and thrive in their careers. We envision a world where every senior engineering leader has access to the knowledge, resources, and support they need to succeed and make a meaningful impact. Through our cutting-edge learning and media content, we aim to inspire and guide senior engineering leaders as they navigate the challenges and opportunities of the fast-paced startup and scale-up ecosystem. Our friendly and inclusive community of experts and peers will provide a safe and supportive space for learning, collaboration, and connection. We are committed to helping senior engineering leaders reach their full potential and achieve their goals, both personally and professionally.Community and Slack GroupThousands of senior engineering leaders are now coming together daily in our mentoring community, getting feedback on challenges, asking for recommendations, building networks and supporting one another in one of the industry's toughest roles.CTO Craft ConA twice-yearly conference for senior technology leaders, bringing them into the room with subject matter experts, coaches and leaders of some of the world's most successful engineering teams.CTO CirclesYou\u2019re not alone: join teams of 10 - 12 emerging CTOs and technology leaders monthly over 12 months, facilitated by an experienced CTO Coach, and build skills around leadership, strategy, process, planning and more.One-to-One CTO CoachingSet yourself up for success: Unlock your professional growth and leadership with face-to-face coaching by an experienced CTO, setting goals and identifying obstacles and patterns.Weekly Bytes EventsExpert discussions and panels on a range of subjects, from leadership to strategy to delivery to culture to stress and wellbeing.Drop me a line to find out more!"}, {"position": "CTO Conference Director", "from": "2021-10-01", "to": "2023-05-08", "days": 584, "country": "", "city": "", "description": "We're working on the biggest CTO Craft conference ever - four days of amazing talks and insights from the best engineering leaders and subject matter experts in the world. Hopefully see you there!"}], "organization": "Self-employed", "from": "2017-05-01", "to": "2023-05-08", "days": 2198}, {"items": [{"position": "Investor and Advisor", "from": "2018-02-01", "to": "2023-05-08", "days": 1922, "country": "", "city": "", "description": ""}], "organization": "Sifted", "from": "2018-02-01", "to": "2023-05-08", "days": 1922}, {"items": [{"position": "Technology and Product Strategy Mentor", "from": "2017-10-01", "to": "2023-05-08", "days": 2045, "country": "", "city": "", "description": "Zinc's mission-driven initiatives bring together 50 entrepreneurs each year to form teams and build commercial businesses which tackle social problems in the developed world. I've helped teams in both missions so far by sharing insight on technical challenges and opportunities, making introductions to people in my network, helping business strategy ideation, etc."}], "organization": "ZINC VC", "from": "2017-10-01", "to": "2023-05-08", "days": 2045}, {"items": [{"position": "Curator in Chief", "from": "2016-08-01", "to": "2023-05-08", "days": 2471, "country": "", "city": "", "description": "What makes a good CTO great? Continuous learning.Learn from the best minds in the world, on subjects including leadership, culture, development, hiring, QA, process, data and much more. Curated lovingly by the team at CTO Craft, delivered to your inbox every Monday morning. ARTICLES FROM THOUGHT LEADERSEvery week, we scour the web for the best blog posts, articles, studies and interviews by the best CTOs, tech leads, product managers and engineers in the world.PODCASTS AND VIDEOSWith the current and classic interviews, videos and podcasts on topics including development, culture, recruitment, testing, big data, cybersecurity and more.COMMENTS AND OPINIONSThe best engineering and leadership minds in the world share their views, success stories, philosophies and mistakes regularly - subscribe to see them all. Subscribe today at http://www.techmanagerweekly.com/"}], "organization": "Tech Manager Weekly", "from": "2016-08-01", "to": "2023-05-08", "days": 2471}, {"items": [{"position": "Investor and Advisor", "from": "2016-03-01", "to": "2023-05-08", "days": 2624, "country": "GB", "city": "London", "description": ""}], "organization": "Trint", "from": "2016-03-01", "to": "2023-05-08", "days": 2624}, {"items": [{"position": "Principal Consultant, CTOzone", "from": "2018-05-01", "to": "2019-10-01", "days": 518, "country": "GB", "city": "London", "description": "Saccade's network of freelance CTOs is now part of 101 Ways!Contact us if you're looking for full or part time support from an experienced freelance CTO, no matter what size or stage your business, or what industry you're in. We provide flexible access to skilled technology leaders to startups, scale-ups, VC-backed businesses and established organisations. If you'd like to know more, drop us a line at jamie@101ways.com or andy.skipper@101ways.com"}], "organization": "101 Ways", "from": "2018-05-01", "to": "2019-10-01", "days": 518}, {"items": [{"position": "Interim CTO", "from": "2018-08-01", "to": "2018-10-01", "days": 61, "country": "GB", "city": "London", "description": "Supporting an Engineering and Product team of 19 through a reorganisation to support new top-level strategies and objectives; analysis of core team strengths and legacy issues, building a recruitment plan for the remainder of 2018."}], "organization": "JustPark", "from": "2018-08-01", "to": "2018-10-01", "days": 61}, {"items": [{"position": "Mentor", "from": "2017-03-01", "to": "2018-03-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "Wayra", "from": "2017-03-01", "to": "2018-03-01", "days": 365}, {"items": [{"position": "Seedcamp Mentor", "from": "2016-09-01", "to": "2017-09-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "Seedcamp", "from": "2016-09-01", "to": "2017-09-01", "days": 365}, {"items": [{"position": "Interim CTO", "from": "2015-05-01", "to": "2016-08-01", "days": 458, "country": "GB", "city": "London", "description": ""}], "organization": "SPYSCAPE", "from": "2015-05-01", "to": "2016-08-01", "days": 458}, {"items": [{"position": "Freelance CTO", "from": "2015-04-01", "to": "2015-09-01", "days": 153, "country": "NL", "city": "", "description": "Working with a highly talented team of PHP engineers, applied a structure for improving project management and resource planning to enable the successful launch of a machine learning-based recommendation tool for job marketing. The tool was launched in late 2015 and is now contributing over 50% of the business' revenues.The tool uses data from over 1500 marketing channels and job boards to suggest the most relevant selection for specific roles. Technology used includes PHP with event sourcing and CQRS, along with Elasticsearch and Ruby-based machine learning and collaborative filtering."}], "organization": "VONQ", "from": "2015-04-01", "to": "2015-09-01", "days": 153}, {"items": [{"position": "Freelance CTO", "from": "2015-03-01", "to": "2015-09-01", "days": 184, "country": "GB", "city": "London", "description": "Assisting with a full reimagination of this community for Multiple Sclerosis sufferers around the globe. I have implemented a new set of processes, as well as supporting new team members in planning and executing a nine-month plan to rebuild the website and release two key projects for partners in the NPO world."}], "organization": "Shift.ms", "from": "2015-03-01", "to": "2015-09-01", "days": 184}, {"items": [{"position": "Freelance CTO", "from": "2015-02-01", "to": "2015-09-01", "days": 212, "country": "", "city": "", "description": ""}], "organization": "Threads Styling Consultancy & Fashion Concierge", "from": "2015-02-01", "to": "2015-09-01", "days": 212}, {"items": [{"position": "Freelance CTO", "from": "2015-01-01", "to": "2015-02-01", "days": 31, "country": "", "city": "", "description": ""}], "organization": "IntuDigital", "from": "2015-01-01", "to": "2015-02-01", "days": 31}, {"items": [{"position": "Chief Technology Officer", "from": "2014-07-01", "to": "2014-10-01", "days": 92, "country": "GB", "city": "London", "description": "Joined as part of a large organisational and culture change effort, to reshape the Future Media and Technology section of the organisation, replace key staff lost to a period of massive churn, rework processes and reengage team members, implement better roadmap and staffing strategies.&#149; Managed a team of 45 - 55 engineers, testers, product and project managers, UX designers and IT personnel.&#149; Created new technology strategy, with the objective of improving efficiency and reducing costs throughout the technology teams, and utilising technology to reduce waste across the organisation.&#149; Reengineered a technology budget of \u00a33.5mm to accommodate new organisational objectives and process improvements; this included reworking several major technical partnerships.&#149; Created a new organisational structure for the web technology team, including a resource plan and strategy for improving the talent pool available through widened visibility in the technical and digital ecosystem.&#149; Outlined plans for a new fundraising CRM system to replace existing legacy data warehouse as primary operational data store for donations and campaign analysis. Created a separate team and strategy for improving the quality of data across various organisational departments to facilitate better analysis and reduce effort.&#149; Planned for first official international campaign in Red Nose Day USA, to be broadcast in May 2015, including resource plan for websites and social campaigns, RFP for donations platform, IVR and SMS."}], "organization": "Comic Relief", "from": "2014-07-01", "to": "2014-10-01", "days": 92}, {"items": [{"position": "Freelance CTO", "from": "2013-11-01", "to": "2014-06-01", "days": 212, "country": "GB", "city": "London", "description": "Working on a suite of tools for the generation and delivery of rich online maps for Airports and Cities worldwide. Product roadmap development, resource planning and onsite management of existing development team, across Java, PostGIS, PHP and Leaflet.js, hosted on Rackspace Cloud."}], "organization": "Living Map", "from": "2013-11-01", "to": "2014-06-01", "days": 212}, {"items": [{"position": "Consultant - Technical Team Organisation", "from": "2014-02-01", "to": "2014-04-01", "days": 59, "country": "GB", "city": "London", "description": "Working with the COO, Derek Gannon, I'm investigating the structure of the technical part of the organisation, with a view to potential restructuring. Suggesting improved workload and talent management methodologies, Agile practices and methods of improving team intercommunication.Identifying key hires for the next generation of the Comic Relief technical team, and means of reducing churn.Identifying risks and bottlenecks in current technologies, infrastructure and team layout."}], "organization": "Comic Relief", "from": "2014-02-01", "to": "2014-04-01", "days": 59}, {"items": [{"position": "eCommerce and Supply Chain Technology Consultant", "from": "2014-02-01", "to": "2014-04-01", "days": 59, "country": "", "city": "", "description": "* Analysing current workflows, business processes and technologies * Reporting on inherent risks and bottlenecks* Advising on new systems and key strategic changes with a view to improving scalability"}], "organization": "The Cambridge Satchel Company", "from": "2014-02-01", "to": "2014-04-01", "days": 59}, {"items": [{"position": "eCommerce Internationalisation Consultant, Freelance CTO", "from": "2013-11-01", "to": "2014-03-01", "days": 120, "country": "GB", "city": "London", "description": "Acting as part-time CTO to this provider of cultural and philosophical seminars, books and articles, run by Alain de Botton and Charlotte Neser, as they move to license the brand across the world. Magento, SilverStripe and Netsuite architecture customisation planning and roll-out, and support of the internal team in defining the business strategy for licensing and internationalisation in general."}], "organization": "The School of Life", "from": "2013-11-01", "to": "2014-03-01", "days": 120}, {"items": [{"position": "Magento and OpenERP Implementation Consultant", "from": "2013-09-01", "to": "2014-03-01", "days": 181, "country": "", "city": "", "description": "Selection and ongoing management of an OpenERP implementation, including migration from existing inventory and accountancy platforms. Planning upgrade and testing of existing Magento platform to enable integration with new OpenERP system, including selection of development partners."}], "organization": "Bridgman Garden Furniture", "from": "2013-09-01", "to": "2014-03-01", "days": 181}, {"items": [{"position": "Freelance CTO, CTO Coach", "from": "2013-09-01", "to": "2014-03-01", "days": 181, "country": "GB", "city": "London", "description": "Providing strategic support to the CTO, assistance in team building and process improvement at this digital video production and delivery agency"}], "organization": "Adjust Your Set", "from": "2013-09-01", "to": "2014-03-01", "days": 181}, {"items": [{"position": "Freelance CTO", "from": "2013-05-01", "to": "2014-01-01", "days": 245, "country": "GB", "city": "London", "description": "Providing ongoing part time technical and operational guidance to this new photographic print retailer. * Complete project turnaround and replacement of Magento development partner* Project scoping and budgeting with founders* Project management and ongoing UAT and advisory* Complete site theme redevelopment, resulting in 110% improvement in Conversion Rate, 15% reduction in bounce rate, prior to an extensive marketing and PR campaign"}], "organization": "Lumitrix", "from": "2013-05-01", "to": "2014-01-01", "days": 245}, {"items": [{"position": "Freelance CTO", "from": "2013-06-01", "to": "2013-08-01", "days": 61, "country": "GB", "city": "London", "description": "Engaged to identify key quality issues in the DIA Boutique Magento site, and manage outsourced development partners to resolve all of the above."}], "organization": "DIA Boutique", "from": "2013-06-01", "to": "2013-08-01", "days": 61}, {"items": [{"position": "Consultant CTO", "from": "2013-06-01", "to": "2013-07-01", "days": 30, "country": "GB", "city": "London", "description": "Working with the founders of this exciting eCommerce business within the pregnancy market.* Definition of Magento project* Selection of Magento development partners* UX design and marketing advisory"}], "organization": "Not yet launched - pregnancy market", "from": "2013-06-01", "to": "2013-07-01", "days": 30}, {"items": [{"position": "Chief Technology Officer", "from": "2010-02-01", "to": "2013-04-01", "days": 1155, "country": "GB", "city": "London", "description": "A multi-award winning online furniture retailer launched in March 2010, backed by a group of investors including Brent Hoberman of Founders Forum and Marc Simoncini of Jaina Capital. The website itself is built using the Magento framework for PHP (built on Zend Framework), interoperating with a Python-based ERP (OpenERP, heavily customised) for stock, dispatch, order and supply chain management. He has direct experience in the logical design of ecommerce and fulfilment systems, as well as online marketing including paid search, retargeting, search engine optimisation and affiliate marketing. All recruitment for each of the technical teams involved was performed by him personally. Also responsible for selection and orchestration of a virtualised hosting environment based on an open-source stack: Nginx, PHP-FPM, MySQL, HAProxy, Memcached, Redis, Puppet, Zabbix, etc. This includes disaster recovery provision, high availability, and the ability to expand the number of servers in the space of minutes in the instance of a sudden burst in traffic.Responsible for all strategic decision making regarding technology within Made.com, including technical team structure and recruiting, site technology selection, office technology and setup across multiple regions and selection of key development and support partners."}], "organization": "MADE.COM", "from": "2010-02-01", "to": "2013-04-01", "days": 1155}, {"items": [{"position": "CTO and Co- founder", "from": "2009-02-01", "to": "2009-12-01", "days": 303, "country": "GB", "city": "London", "description": "Social media startup within a specific vertical - Django-based stack"}], "organization": "Unnamed", "from": "2009-02-01", "to": "2009-12-01", "days": 303}, {"items": [{"position": "CTO and Lead Architect", "from": "2006-11-01", "to": "2009-01-01", "days": 792, "country": "GB", "city": "London", "description": "Responsible for all aspects of technical vision within the company, development of the business's core product and local infrastructure maintenance. Management of a multi-million pound IT budget in this venture capital backed startup online business, also responsible for strategic early-stage planning of the core functionality offered by the site prior to its development. Architecture of an ASP.Net framework to support a rich planner application for home-movers in the UK.  Board member. * Recruited and led the technical, creative and project management team - up to ten employees * Led all architectural decisions during development, selection of technologies, made build-vs-buy decisions * Aligned IT with business goals to minimise cost and ensure a solid foundation for the lifetime of the company * Implemented Agile working practices throughout the company (Scrum) to improve collection of specifications, planning and estimation accuracy * Created Quality Assurance processes to ensure acceptance of the software by business stakeholders and the public at large * Scoped and led the development of a white-labelled system to aid partnerships with several large multinational corporations, eg MSN, Yahoo, AOL, Nectar * Infrastructure design and management, technology provider sourcing and overseeing of the server setup and deployment"}], "organization": "moveme.com", "from": "2006-11-01", "to": "2009-01-01", "days": 792}, {"items": [{"position": "Senior Developer & Architect", "from": "2005-01-01", "to": "2006-04-01", "days": 455, "country": "GB", "city": "London", "description": "Lead developer for a cross-brand promotional campaign for Proctor & Gamble to coincide with the Fifa World Cup, including a fully-featured content management system for a Flash game which was fully localised for 30+ regions and languages."}], "organization": "Skive Creative", "from": "2005-01-01", "to": "2006-04-01", "days": 455}, {"items": [{"position": "Web developer", "from": "2005-01-01", "to": "2006-01-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "Freelancing", "from": "2005-01-01", "to": "2006-01-01", "days": 365}, {"items": [{"position": "Developer", "from": "2001-01-01", "to": "2004-01-01", "days": 1095, "country": "", "city": "", "description": ""}], "organization": "Paradisii Ltd", "from": "2001-01-01", "to": "2004-01-01", "days": 1095}], "education": [{"items": [{"degree": "", "from": "", "to": "", "days": "", "country": "", "city": "", "description": ""}], "organization": "", "from": "", "to": "", "days": ""}]})
l.append({"info": {"name": "Andy Skipper", "full_name": "SKIPPER, Andy", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "", "city": ""}, "experience": [{"items": [{"position": "Founder and Chief CTO Coach", "from": "2017-05-01", "to": "2023-05-08", "days": 2198, "country": "GB", "city": "London", "description": "Our mission is to empower senior engineering leaders in startups and scale-ups to continuously learn, grow, and thrive in their careers. We envision a world where every senior engineering leader has access to the knowledge, resources, and support they need to succeed and make a meaningful impact. Through our cutting-edge learning and media content, we aim to inspire and guide senior engineering leaders as they navigate the challenges and opportunities of the fast-paced startup and scale-up ecosystem. Our friendly and inclusive community of experts and peers will provide a safe and supportive space for learning, collaboration, and connection. We are committed to helping senior engineering leaders reach their full potential and achieve their goals, both personally and professionally.Community and Slack GroupThousands of senior engineering leaders are now coming together daily in our mentoring community, getting feedback on challenges, asking for recommendations, building networks and supporting one another in one of the industry's toughest roles.CTO Craft ConA twice-yearly conference for senior technology leaders, bringing them into the room with subject matter experts, coaches and leaders of some of the world's most successful engineering teams.CTO CirclesYou\u2019re not alone: join teams of 10 - 12 emerging CTOs and technology leaders monthly over 12 months, facilitated by an experienced CTO Coach, and build skills around leadership, strategy, process, planning and more.One-to-One CTO CoachingSet yourself up for success: Unlock your professional growth and leadership with face-to-face coaching by an experienced CTO, setting goals and identifying obstacles and patterns.Weekly Bytes EventsExpert discussions and panels on a range of subjects, from leadership to strategy to delivery to culture to stress and wellbeing.Drop me a line to find out more!"}, {"position": "CTO Conference Director", "from": "2021-10-01", "to": "2023-05-08", "days": 584, "country": "", "city": "", "description": "We're working on the biggest CTO Craft conference ever - four days of amazing talks and insights from the best engineering leaders and subject matter experts in the world. Hopefully see you there!"}], "organization": "Self-employed", "from": "2017-05-01", "to": "2023-05-08", "days": 2198}, {"items": [{"position": "Investor and Advisor", "from": "2018-02-01", "to": "2023-05-08", "days": 1922, "country": "", "city": "", "description": ""}], "organization": "Sifted", "from": "2018-02-01", "to": "2023-05-08", "days": 1922}, {"items": [{"position": "Technology and Product Strategy Mentor", "from": "2017-10-01", "to": "2023-05-08", "days": 2045, "country": "", "city": "", "description": "Zinc's mission-driven initiatives bring together 50 entrepreneurs each year to form teams and build commercial businesses which tackle social problems in the developed world. I've helped teams in both missions so far by sharing insight on technical challenges and opportunities, making introductions to people in my network, helping business strategy ideation, etc."}], "organization": "ZINC VC", "from": "2017-10-01", "to": "2023-05-08", "days": 2045}, {"items": [{"position": "Curator in Chief", "from": "2016-08-01", "to": "2023-05-08", "days": 2471, "country": "", "city": "", "description": "What makes a good CTO great? Continuous learning.Learn from the best minds in the world, on subjects including leadership, culture, development, hiring, QA, process, data and much more. Curated lovingly by the team at CTO Craft, delivered to your inbox every Monday morning. ARTICLES FROM THOUGHT LEADERSEvery week, we scour the web for the best blog posts, articles, studies and interviews by the best CTOs, tech leads, product managers and engineers in the world.PODCASTS AND VIDEOSWith the current and classic interviews, videos and podcasts on topics including development, culture, recruitment, testing, big data, cybersecurity and more.COMMENTS AND OPINIONSThe best engineering and leadership minds in the world share their views, success stories, philosophies and mistakes regularly - subscribe to see them all. Subscribe today at http://www.techmanagerweekly.com/"}], "organization": "Tech Manager Weekly", "from": "2016-08-01", "to": "2023-05-08", "days": 2471}, {"items": [{"position": "Investor and Advisor", "from": "2016-03-01", "to": "2023-05-08", "days": 2624, "country": "GB", "city": "London", "description": ""}], "organization": "Trint", "from": "2016-03-01", "to": "2023-05-08", "days": 2624}, {"items": [{"position": "Principal Consultant, CTOzone", "from": "2018-05-01", "to": "2019-10-01", "days": 518, "country": "GB", "city": "London", "description": "Saccade's network of freelance CTOs is now part of 101 Ways!Contact us if you're looking for full or part time support from an experienced freelance CTO, no matter what size or stage your business, or what industry you're in. We provide flexible access to skilled technology leaders to startups, scale-ups, VC-backed businesses and established organisations. If you'd like to know more, drop us a line at jamie@101ways.com or andy.skipper@101ways.com"}], "organization": "101 Ways", "from": "2018-05-01", "to": "2019-10-01", "days": 518}, {"items": [{"position": "Interim CTO", "from": "2018-08-01", "to": "2018-10-01", "days": 61, "country": "GB", "city": "London", "description": "Supporting an Engineering and Product team of 19 through a reorganisation to support new top-level strategies and objectives; analysis of core team strengths and legacy issues, building a recruitment plan for the remainder of 2018."}], "organization": "JustPark", "from": "2018-08-01", "to": "2018-10-01", "days": 61}, {"items": [{"position": "Mentor", "from": "2017-03-01", "to": "2018-03-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "Wayra", "from": "2017-03-01", "to": "2018-03-01", "days": 365}, {"items": [{"position": "Seedcamp Mentor", "from": "2016-09-01", "to": "2017-09-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "Seedcamp", "from": "2016-09-01", "to": "2017-09-01", "days": 365}, {"items": [{"position": "Interim CTO", "from": "2015-05-01", "to": "2016-08-01", "days": 458, "country": "GB", "city": "London", "description": ""}], "organization": "SPYSCAPE", "from": "2015-05-01", "to": "2016-08-01", "days": 458}, {"items": [{"position": "Freelance CTO", "from": "2015-04-01", "to": "2015-09-01", "days": 153, "country": "NL", "city": "", "description": "Working with a highly talented team of PHP engineers, applied a structure for improving project management and resource planning to enable the successful launch of a machine learning-based recommendation tool for job marketing. The tool was launched in late 2015 and is now contributing over 50% of the business' revenues.The tool uses data from over 1500 marketing channels and job boards to suggest the most relevant selection for specific roles. Technology used includes PHP with event sourcing and CQRS, along with Elasticsearch and Ruby-based machine learning and collaborative filtering."}], "organization": "VONQ", "from": "2015-04-01", "to": "2015-09-01", "days": 153}, {"items": [{"position": "Freelance CTO", "from": "2015-03-01", "to": "2015-09-01", "days": 184, "country": "GB", "city": "London", "description": "Assisting with a full reimagination of this community for Multiple Sclerosis sufferers around the globe. I have implemented a new set of processes, as well as supporting new team members in planning and executing a nine-month plan to rebuild the website and release two key projects for partners in the NPO world."}], "organization": "Shift.ms", "from": "2015-03-01", "to": "2015-09-01", "days": 184}, {"items": [{"position": "Freelance CTO", "from": "2015-02-01", "to": "2015-09-01", "days": 212, "country": "", "city": "", "description": ""}], "organization": "Threads Styling Consultancy & Fashion Concierge", "from": "2015-02-01", "to": "2015-09-01", "days": 212}, {"items": [{"position": "Freelance CTO", "from": "2015-01-01", "to": "2015-02-01", "days": 31, "country": "", "city": "", "description": ""}], "organization": "IntuDigital", "from": "2015-01-01", "to": "2015-02-01", "days": 31}, {"items": [{"position": "Chief Technology Officer", "from": "2014-07-01", "to": "2014-10-01", "days": 92, "country": "GB", "city": "London", "description": "Joined as part of a large organisational and culture change effort, to reshape the Future Media and Technology section of the organisation, replace key staff lost to a period of massive churn, rework processes and reengage team members, implement better roadmap and staffing strategies.&#149; Managed a team of 45 - 55 engineers, testers, product and project managers, UX designers and IT personnel.&#149; Created new technology strategy, with the objective of improving efficiency and reducing costs throughout the technology teams, and utilising technology to reduce waste across the organisation.&#149; Reengineered a technology budget of \u00a33.5mm to accommodate new organisational objectives and process improvements; this included reworking several major technical partnerships.&#149; Created a new organisational structure for the web technology team, including a resource plan and strategy for improving the talent pool available through widened visibility in the technical and digital ecosystem.&#149; Outlined plans for a new fundraising CRM system to replace existing legacy data warehouse as primary operational data store for donations and campaign analysis. Created a separate team and strategy for improving the quality of data across various organisational departments to facilitate better analysis and reduce effort.&#149; Planned for first official international campaign in Red Nose Day USA, to be broadcast in May 2015, including resource plan for websites and social campaigns, RFP for donations platform, IVR and SMS."}], "organization": "Comic Relief", "from": "2014-07-01", "to": "2014-10-01", "days": 92}, {"items": [{"position": "Freelance CTO", "from": "2013-11-01", "to": "2014-06-01", "days": 212, "country": "GB", "city": "London", "description": "Working on a suite of tools for the generation and delivery of rich online maps for Airports and Cities worldwide. Product roadmap development, resource planning and onsite management of existing development team, across Java, PostGIS, PHP and Leaflet.js, hosted on Rackspace Cloud."}], "organization": "Living Map", "from": "2013-11-01", "to": "2014-06-01", "days": 212}, {"items": [{"position": "Consultant - Technical Team Organisation", "from": "2014-02-01", "to": "2014-04-01", "days": 59, "country": "GB", "city": "London", "description": "Working with the COO, Derek Gannon, I'm investigating the structure of the technical part of the organisation, with a view to potential restructuring. Suggesting improved workload and talent management methodologies, Agile practices and methods of improving team intercommunication.Identifying key hires for the next generation of the Comic Relief technical team, and means of reducing churn.Identifying risks and bottlenecks in current technologies, infrastructure and team layout."}], "organization": "Comic Relief", "from": "2014-02-01", "to": "2014-04-01", "days": 59}, {"items": [{"position": "eCommerce and Supply Chain Technology Consultant", "from": "2014-02-01", "to": "2014-04-01", "days": 59, "country": "", "city": "", "description": "* Analysing current workflows, business processes and technologies * Reporting on inherent risks and bottlenecks* Advising on new systems and key strategic changes with a view to improving scalability"}], "organization": "The Cambridge Satchel Company", "from": "2014-02-01", "to": "2014-04-01", "days": 59}, {"items": [{"position": "eCommerce Internationalisation Consultant, Freelance CTO", "from": "2013-11-01", "to": "2014-03-01", "days": 120, "country": "GB", "city": "London", "description": "Acting as part-time CTO to this provider of cultural and philosophical seminars, books and articles, run by Alain de Botton and Charlotte Neser, as they move to license the brand across the world. Magento, SilverStripe and Netsuite architecture customisation planning and roll-out, and support of the internal team in defining the business strategy for licensing and internationalisation in general."}], "organization": "The School of Life", "from": "2013-11-01", "to": "2014-03-01", "days": 120}, {"items": [{"position": "Magento and OpenERP Implementation Consultant", "from": "2013-09-01", "to": "2014-03-01", "days": 181, "country": "", "city": "", "description": "Selection and ongoing management of an OpenERP implementation, including migration from existing inventory and accountancy platforms. Planning upgrade and testing of existing Magento platform to enable integration with new OpenERP system, including selection of development partners."}], "organization": "Bridgman Garden Furniture", "from": "2013-09-01", "to": "2014-03-01", "days": 181}, {"items": [{"position": "Freelance CTO, CTO Coach", "from": "2013-09-01", "to": "2014-03-01", "days": 181, "country": "GB", "city": "London", "description": "Providing strategic support to the CTO, assistance in team building and process improvement at this digital video production and delivery agency"}], "organization": "Adjust Your Set", "from": "2013-09-01", "to": "2014-03-01", "days": 181}, {"items": [{"position": "Freelance CTO", "from": "2013-05-01", "to": "2014-01-01", "days": 245, "country": "GB", "city": "London", "description": "Providing ongoing part time technical and operational guidance to this new photographic print retailer. * Complete project turnaround and replacement of Magento development partner* Project scoping and budgeting with founders* Project management and ongoing UAT and advisory* Complete site theme redevelopment, resulting in 110% improvement in Conversion Rate, 15% reduction in bounce rate, prior to an extensive marketing and PR campaign"}], "organization": "Lumitrix", "from": "2013-05-01", "to": "2014-01-01", "days": 245}, {"items": [{"position": "Freelance CTO", "from": "2013-06-01", "to": "2013-08-01", "days": 61, "country": "GB", "city": "London", "description": "Engaged to identify key quality issues in the DIA Boutique Magento site, and manage outsourced development partners to resolve all of the above."}], "organization": "DIA Boutique", "from": "2013-06-01", "to": "2013-08-01", "days": 61}, {"items": [{"position": "Consultant CTO", "from": "2013-06-01", "to": "2013-07-01", "days": 30, "country": "GB", "city": "London", "description": "Working with the founders of this exciting eCommerce business within the pregnancy market.* Definition of Magento project* Selection of Magento development partners* UX design and marketing advisory"}], "organization": "Not yet launched - pregnancy market", "from": "2013-06-01", "to": "2013-07-01", "days": 30}, {"items": [{"position": "Chief Technology Officer", "from": "2010-02-01", "to": "2013-04-01", "days": 1155, "country": "GB", "city": "London", "description": "A multi-award winning online furniture retailer launched in March 2010, backed by a group of investors including Brent Hoberman of Founders Forum and Marc Simoncini of Jaina Capital. The website itself is built using the Magento framework for PHP (built on Zend Framework), interoperating with a Python-based ERP (OpenERP, heavily customised) for stock, dispatch, order and supply chain management. He has direct experience in the logical design of ecommerce and fulfilment systems, as well as online marketing including paid search, retargeting, search engine optimisation and affiliate marketing. All recruitment for each of the technical teams involved was performed by him personally. Also responsible for selection and orchestration of a virtualised hosting environment based on an open-source stack: Nginx, PHP-FPM, MySQL, HAProxy, Memcached, Redis, Puppet, Zabbix, etc. This includes disaster recovery provision, high availability, and the ability to expand the number of servers in the space of minutes in the instance of a sudden burst in traffic.Responsible for all strategic decision making regarding technology within Made.com, including technical team structure and recruiting, site technology selection, office technology and setup across multiple regions and selection of key development and support partners."}], "organization": "MADE.COM", "from": "2010-02-01", "to": "2013-04-01", "days": 1155}, {"items": [{"position": "CTO and Co- founder", "from": "2009-02-01", "to": "2009-12-01", "days": 303, "country": "GB", "city": "London", "description": "Social media startup within a specific vertical - Django-based stack"}], "organization": "Unnamed", "from": "2009-02-01", "to": "2009-12-01", "days": 303}, {"items": [{"position": "CTO and Lead Architect", "from": "2006-11-01", "to": "2009-01-01", "days": 792, "country": "GB", "city": "London", "description": "Responsible for all aspects of technical vision within the company, development of the business's core product and local infrastructure maintenance. Management of a multi-million pound IT budget in this venture capital backed startup online business, also responsible for strategic early-stage planning of the core functionality offered by the site prior to its development. Architecture of an ASP.Net framework to support a rich planner application for home-movers in the UK.  Board member. * Recruited and led the technical, creative and project management team - up to ten employees * Led all architectural decisions during development, selection of technologies, made build-vs-buy decisions * Aligned IT with business goals to minimise cost and ensure a solid foundation for the lifetime of the company * Implemented Agile working practices throughout the company (Scrum) to improve collection of specifications, planning and estimation accuracy * Created Quality Assurance processes to ensure acceptance of the software by business stakeholders and the public at large * Scoped and led the development of a white-labelled system to aid partnerships with several large multinational corporations, eg MSN, Yahoo, AOL, Nectar * Infrastructure design and management, technology provider sourcing and overseeing of the server setup and deployment"}], "organization": "moveme.com", "from": "2006-11-01", "to": "2009-01-01", "days": 792}, {"items": [{"position": "Senior Developer & Architect", "from": "2005-01-01", "to": "2006-04-01", "days": 455, "country": "GB", "city": "London", "description": "Lead developer for a cross-brand promotional campaign for Proctor & Gamble to coincide with the Fifa World Cup, including a fully-featured content management system for a Flash game which was fully localised for 30+ regions and languages."}], "organization": "Skive Creative", "from": "2005-01-01", "to": "2006-04-01", "days": 455}, {"items": [{"position": "Web developer", "from": "2005-01-01", "to": "2006-01-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "Freelancing", "from": "2005-01-01", "to": "2006-01-01", "days": 365}, {"items": [{"position": "Developer", "from": "2001-01-01", "to": "2004-01-01", "days": 1095, "country": "", "city": "", "description": ""}], "organization": "Paradisii Ltd", "from": "2001-01-01", "to": "2004-01-01", "days": 1095}], "education": [{"items": [{"degree": "", "from": "", "to": "", "days": "", "country": "", "city": "", "description": ""}], "organization": "", "from": "", "to": "", "days": ""}]})
l.append({"info": {"name": "Nicola Thompson", "full_name": "THOMPSON, Nicola", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "GB", "city": "London"}, "experience": [{"items": [{"position": "Volunteer Advisor", "from": "2023-03-01", "to": "2023-05-08", "days": 68, "country": "GB", "city": "", "description": "Volunteer advisor to CEOs of U.K. SMEs as part of the All together network"}], "organization": "All Together", "from": "2023-03-01", "to": "2023-05-08", "days": 68}, {"items": [{"position": "Board Advisor", "from": "2020-05-01", "to": "2023-05-08", "days": 1102, "country": "GB", "city": "London", "description": "Skills: Business Planning"}], "organization": "Various Startups", "from": "2020-05-01", "to": "2023-05-08", "days": 1102}, {"items": [{"position": "Chief Executive Officer", "from": "2022-03-01", "to": "2022-12-01", "days": 275, "country": "", "city": "", "description": "Skills: Strategy Implementation"}, {"position": "Chief Operating Officer at MADE.com", "from": "2019-10-01", "to": "2022-02-01", "days": 854, "country": "GB", "city": "London", "description": "Skills: Strategy Implementation"}], "organization": "MADE.COM", "from": "2019-10-01", "to": "2022-12-01", "days": 1157}, {"items": [{"position": "Global Customer Development Director", "from": "2019-02-01", "to": "2019-09-01", "days": 212, "country": "GB", "city": "London", "description": "Skills: Strategy Implementation"}, {"position": "Global Trade Director", "from": "2016-05-01", "to": "2019-02-01", "days": 1006, "country": "GB", "city": "London", "description": "Skills: Business Development"}, {"position": "Merchandise Director", "from": "2015-05-01", "to": "2016-05-01", "days": 366, "country": "", "city": "", "description": "Skills: Annual Budgeting"}, {"position": "Head of Merchandising", "from": "2014-04-01", "to": "2015-05-01", "days": 395, "country": "", "city": "", "description": "\u2022 Developed and grew the key sports footwear categories and the approach to Face and Body\u2022 Worked with rest of womenswear management team to define business approach to merchandising in fast growing business"}], "organization": "ASOS.com", "from": "2014-04-01", "to": "2019-09-01", "days": 1979}, {"items": [{"position": "Commerical Director", "from": "2012-05-01", "to": "2014-04-01", "days": 700, "country": "GB", "city": "London", "description": "\u2022 Part of the founding team of a fast growing fashion ecommerce start up \u2022 Creating take to market business strategy, product roadmap, paid marketing strategy, operational planning and building a founding team from scratch\u2022 Strong experience in Investor relations, raising capital, venture capital requirements."}], "organization": "Atterley Road", "from": "2012-05-01", "to": "2014-04-01", "days": 700}, {"items": [{"position": "Ecommerce and Wholesale Manager", "from": "2011-05-01", "to": "2012-05-01", "days": 366, "country": "", "city": "", "description": ""}, {"position": "Senior Merchandiser", "from": "2008-08-01", "to": "2011-05-01", "days": 1003, "country": "GB", "city": "London", "description": ""}], "organization": "WHISTLES", "from": "2008-08-01", "to": "2012-05-01", "days": 1369}, {"items": [{"position": "Ecommerce and core merchandising", "from": "2002-01-01", "to": "2008-01-01", "days": 2191, "country": "", "city": "", "description": ""}], "organization": "TOPSHOP TOPMAN", "from": "2002-01-01", "to": "2008-01-01", "days": 2191}, {"items": [{"position": "Graduate trainee and junior management roles", "from": "1998-09-01", "to": "2002-01-01", "days": 1218, "country": "", "city": "", "description": ""}], "organization": "Arcadia Group Ltd", "from": "1998-09-01", "to": "2002-01-01", "days": 1218}], "education": [{"items": [{"degree": "BSc, Geography", "from": "1995-01-01", "to": "1998-01-01", "days": 1096, "country": "", "city": "", "description": ""}], "organization": "The University of Manchester", "from": "1995-01-01", "to": "1998-01-01", "days": 1096}]})
l.append({"info": {"name": "Lisa Gan Tomlins", "full_name": "TOMLINS, Lisa Gan", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "GB", "city": "London"}, "experience": [{"items": [{"position": "General Counsel & Company Secretary", "from": "2014-12-01", "to": "2022-08-01", "days": 2800, "country": "GB", "city": "London", "description": "Joined as first lawyer, was sole lawyer advising on all commercial/corporate matters for first 4 years, and thereafter scaled the legal/company secretarial function to a team of 9. Guided the company through a Series C fundraising, London IPO and its first year as a public company. Strategic adviser to Board and key member of Exec Team.  Built and led team providing legal support for the entire business across all its markets of operation as it expanded internationally and quadrupled in size.  Delivered strategic risk-based advice on a broad range of both corporate matters (e.g. IPO, M&A/fundraisings/financings, governance/regulations, corporate structure/company secretarial) and commercial matters (e.g. e-commerce/retail, IP/brand/marketing, real estate/supply chain, technology/data protection, litigation/regulatory, employment/share incentives/tax)."}], "organization": "MADE.COM", "from": "2014-12-01", "to": "2022-08-01", "days": 2800}, {"items": [{"position": "Legal Counsel", "from": "2012-06-01", "to": "2014-11-01", "days": 883, "country": "GB", "city": "London", "description": "Led key IPO legal workstreams, enabling successful admission to the London Stock Exchange and \u00a3387 million capital raising in April 2014.Joint responsibility for UK legal matters, and lead responsibility for Canadian legal matters.  Reported directly to the General Counsel.Delivered practical business-focused legal advice on a broad spectrum of B2C and B2B matters, including:  M&A, marketing partnerships, technology projects, IP protection and dispute resolution."}], "organization": "Just-Eat Group", "from": "2012-06-01", "to": "2014-11-01", "days": 883}, {"items": [{"position": "Senior Associate", "from": "2006-08-01", "to": "2011-05-01", "days": 1734, "country": "GB", "city": "London", "description": "Advised companies and financial institutions on numerous IPOs, rights offerings, bond offerings, M&A transactions and other high-value capital market and corporate transactions.  As a dual-qualified lawyer, uniquely positioned to advise on securities laws and corporate matters and cross-border transactions from both a US and UK perspective."}], "organization": "Cleary Gottlieb Steen & Hamilton LLP", "from": "2006-08-01", "to": "2011-05-01", "days": 1734}, {"items": [{"position": "Legal Counsel (Secondee), Investment Banking Department", "from": "2010-07-01", "to": "2011-01-01", "days": 184, "country": "GB", "city": "London", "description": "Advised banking teams on a broad range of investment banking transactions, including IPOs, rights offerings, bond offerings and M&A transactions."}], "organization": "Goldman Sachs", "from": "2010-07-01", "to": "2011-01-01", "days": 184}, {"items": [{"position": "Associate", "from": "2003-08-01", "to": "2006-06-01", "days": 1035, "country": "GB", "city": "London", "description": "Advised companies and financial institutions on numerous M&A, restructuring, finance and capital market transactions.Provided pro bono legal advice at the Hackney Community Law Centre in East London."}, {"position": "Associate", "from": "2002-08-01", "to": "2003-07-01", "days": 334, "country": "", "city": "", "description": ""}], "organization": "Debevoise & Plimpton", "from": "2002-08-01", "to": "2006-06-01", "days": 1400}], "education": [{"items": [{"degree": "B.A. (Hons), First Class Honours, Law", "from": "1998-01-01", "to": "2001-01-01", "days": 1096, "country": "", "city": "", "description": ""}], "organization": "University of Cambridge", "from": "1998-01-01", "to": "2001-01-01", "days": 1096}, {"items": [{"degree": "LL.M. (Masters of Law), Law", "from": "2001-01-01", "to": "2002-01-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "New York University School of Law", "from": "2001-01-01", "to": "2002-01-01", "days": 365}]})
l.append({"info": {"name": "Laura Tyler", "full_name": "TYLER, Laura", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "GB", "city": "London"}, "experience": [{"items": [{"position": "General Counsel & Company Secretary", "from": "2022-07-01", "to": "2023-02-01", "days": 215, "country": "GB", "city": "London", "description": ""}, {"position": "Head of Legal (Corporate)", "from": "2022-04-01", "to": "2022-08-01", "days": 122, "country": "GB", "city": "London", "description": ""}, {"position": "Senior Legal Counsel", "from": "2022-01-01", "to": "2022-04-01", "days": 90, "country": "", "city": "", "description": ""}, {"position": "Legal Counsel", "from": "2020-07-01", "to": "2022-02-01", "days": 580, "country": "GB", "city": "London", "description": ""}], "organization": "MADE.COM", "from": "2020-07-01", "to": "2023-02-01", "days": 945}, {"items": [{"position": "Corporate Associate", "from": "2019-02-01", "to": "2020-07-01", "days": 516, "country": "GB", "city": "London", "description": "I was an associate in Bird & Bird's Corporate group where I gained experience in a range of corporate matters with a focus on venture capital investments, acting for both companies and investors."}], "organization": "Bird & Bird", "from": "2019-02-01", "to": "2020-07-01", "days": 516}, {"items": [{"position": "Corporate Associate", "from": "2017-03-01", "to": "2018-11-01", "days": 610, "country": "GB", "city": "London", "description": "Associate - Corporate"}, {"position": "Trainee solicitor", "from": "2015-03-01", "to": "2017-03-01", "days": 731, "country": "GB", "city": "London", "description": "First seat - Real EstateSecond seat - Commercial DisputesThird seat - Corporate (Private and Public M&A, ECM)Fourth seat - Corporate (M&A - secondment to HSF Singapore / Jakarta)"}], "organization": "Herbert Smith Freehills", "from": "2015-03-01", "to": "2018-11-01", "days": 1341}, {"items": [{"position": "Legal Counsel (Secondment)", "from": "2017-06-01", "to": "2017-11-01", "days": 153, "country": "GB", "city": "London", "description": ""}], "organization": "ClearBank\u00ae", "from": "2017-06-01", "to": "2017-11-01", "days": 153}, {"items": [{"position": "Business Management Intern", "from": "2011-09-01", "to": "2012-06-01", "days": 274, "country": "FR", "city": "Paris", "description": ""}], "organization": "Freshfields Bruckhaus Deringer", "from": "2011-09-01", "to": "2012-06-01", "days": 274}], "education": [{"items": [{"degree": "", "from": "2013-01-01", "to": "2015-01-01", "days": 730, "country": "", "city": "", "description": ""}], "organization": "BPP Law School", "from": "2013-01-01", "to": "2015-01-01", "days": 730}, {"items": [{"degree": "Bachelor of Arts (B.A.) Hons, French and Spanish", "from": "2009-01-01", "to": "2013-01-01", "days": 1461, "country": "", "city": "", "description": ""}], "organization": "University of Cambridge", "from": "2009-01-01", "to": "2013-01-01", "days": 1461}]})
l.append({"info": {"name": "David Vanek", "full_name": "VANEK, David", "legal_company_name": "MADE.COM DESIGN LTD", "company_id": "07101408", "country": "GB", "city": "London"}, "experience": [{"items": [{"position": "Co-Founder and CEO", "from": "2016-11-01", "to": "2023-05-08", "days": 2379, "country": "GB", "city": "London", "description": ""}], "organization": "Anorak Technologies Ltd", "from": "2016-11-01", "to": "2023-05-08", "days": 2379}, {"items": [{"position": "Entrepreneur", "from": "2016-11-01", "to": "2023-05-08", "days": 2379, "country": "GB", "city": "London", "description": ""}], "organization": "Kamet", "from": "2016-11-01", "to": "2023-05-08", "days": 2379}, {"items": [{"position": "Investor and Board Member", "from": "2013-03-01", "to": "2023-05-08", "days": 3720, "country": "FR", "city": "", "description": ""}], "organization": "MAISON STANDARDS", "from": "2013-03-01", "to": "2023-05-08", "days": 3720}, {"items": [{"position": "Board member and co-founder", "from": "2006-01-01", "to": "2017-07-01", "days": 4199, "country": "FR", "city": "", "description": "SweetHome / BienServi group is a leading player in the French Homeservices industry. Serving more than 1,000 clients with 200 employees, BienServi provides housekeeping and handyman services."}], "organization": "SweetHomme Service Group", "from": "2006-01-01", "to": "2017-07-01", "days": 4199}, {"items": [{"position": "Chief Financial Officer", "from": "2015-09-01", "to": "2016-11-01", "days": 427, "country": "", "city": "", "description": ""}, {"position": "Director of Business Development and Strategic Projects", "from": "2014-01-01", "to": "2015-08-01", "days": 577, "country": "", "city": "", "description": ""}, {"position": "General Manager France & Italy", "from": "2012-08-01", "to": "2013-12-01", "days": 487, "country": "", "city": "", "description": ""}], "organization": "MADE.COM", "from": "2012-08-01", "to": "2016-11-01", "days": 1553}, {"items": [{"position": "CEO and Co-founder", "from": "2009-01-01", "to": "2012-06-01", "days": 1247, "country": "FR", "city": "", "description": "Pioneer on the French daily deal market. Built one of the leading platform with 700,000 members and more than 50,000 buyersRecruitment and management of a 45 people team off the ground (sales rep, webmarketing, etc.). Business development and strategic partnerships negotiation (social and media partnerships with media groups, fund raising, etc.)"}], "organization": "Bon-prive.com", "from": "2009-01-01", "to": "2012-06-01", "days": 1247}, {"items": [{"position": "Vice President - M&A/ECM - TMT Sector", "from": "2001-01-01", "to": "2009-01-01", "days": 2922, "country": "FR", "city": "", "description": "-Corporate Finance advisory in both M&A and ECM transactions in continental Europe-M&A advisory: Involvement in advisory works for cross-border transactions in various industries (eTravel, ecommerce, social media, etc.) : Meetic, Priceminister, Viadeo, etc.-ECM advisory: Pro-active role in IPO and Capital Increase for leading European Internet players"}], "organization": "J.P. Morgan", "from": "2001-01-01", "to": "2009-01-01", "days": 2922}, {"items": [{"position": "Associate", "from": "2000-01-01", "to": "2002-01-01", "days": 731, "country": "FR", "city": "", "description": "Research and economic intelligence on TMT sectorsCompany valuation, financial impact modelling, deal structuringSelection process : business plan analysis, entrepreneurs and projects assessment, etc.Portfolio follow-up : corporate finance issues (M&A, disposal), operational coaching, etc."}], "organization": "Angel-Invest", "from": "2000-01-01", "to": "2002-01-01", "days": 731}, {"items": [{"position": "Analyst", "from": "1999-01-01", "to": "2000-01-01", "days": 365, "country": "FR", "city": "", "description": ""}], "organization": "Schroders Salomon Smith Barney", "from": "1999-01-01", "to": "2000-01-01", "days": 365}], "education": [{"items": [{"degree": "Specialised Master Degree, Corporate Finance", "from": "1998-01-01", "to": "1999-01-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "emlyon business school", "from": "1998-01-01", "to": "1999-01-01", "days": 365}, {"items": [{"degree": "Master Degree, Finance & Management", "from": "1993-01-01", "to": "1997-01-01", "days": 1461, "country": "", "city": "", "description": ""}], "organization": "Universit\u00e9 Paris Dauphine - PSL", "from": "1993-01-01", "to": "1997-01-01", "days": 1461}]})


In [247]:
# find indices with same linked in name
def remove_duplicates(data):
    l = data
    same_indices = []

    for i, item in enumerate(l):
        indices = [j for j, x in enumerate(l) if x['info']['name'] == item['info']['name']]
        if len(indices) > 1 and indices not in same_indices:
            same_indices.append(indices)

#     print(same_indices)
    
    # find the company house name that most closely matches linked in names and remove the other duplicates
    remove_indices = []
    for same_index in same_indices:

        max_score = 0
        for same_sub_index in same_index:
            score = fuzz.token_sort_ratio(l[same_sub_index]['info']['name'], l[same_sub_index]['info']['full_name'])
            if score >= max_score:
                keep_index = same_sub_index
                max_score = score
        remove_index = same_index.copy()
        remove_index.remove(keep_index)
        remove_indices.extend(remove_index)
#         print('keep index', keep_index, 'from', same_index, 'remove index', remove_index)
        
    # remove duplicates
    remove_indices.sort(reverse=True)  # Sort in descending order to avoid index shifting

    for index in remove_indices:
        l.pop(index)

    return  l

In [250]:
remove_duplicates(l)

[{'info': {'name': 'Carson Roen',
   'full_name': 'ROEN, Carson',
   'legal_company_name': 'MADE.COM DESIGN LTD',
   'company_id': '07101408',
   'country': 'GB',
   'city': 'London'},
  'experience': [{'items': [{'position': 'Marketing',
      'from': '2017-01-01',
      'to': '2023-05-08',
      'days': 2318,
      'country': 'GB',
      'city': 'London',
      'description': ''}],
    'organization': 'Strike',
    'from': '2017-01-01',
    'to': '2023-05-08',
    'days': 2318},
   {'items': [{'position': 'Marketing',
      'from': '2013-10-01',
      'to': '2016-02-01',
      'days': 853,
      'country': 'GB',
      'city': 'London',
      'description': 'Performance marketing & advertising, measurement, analysis, brand building, market positioning, comms strategy, strategic business development, monetization/optimization and all things marketing to ensure the business continues to grow in a fast, efficient and sustainable way. General management and leadership beyond marketing inc

In [251]:
len(l)

7